In [46]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import ndcg_score
from sklearn.metrics.pairwise import cosine_similarity

In [47]:
ratings = pd.read_csv('../data/interim/u.data', index_col=0)

In [48]:
train, test = train_test_split(ratings.values, test_size=0.2, random_state=0, stratify=ratings.user)
train_df = pd.DataFrame(train, columns=ratings.columns)
test_df = pd.DataFrame(test, columns=ratings.columns)

In [49]:
train_df.head()

,user,item,rating,timestamp
0,90,610,5,891383753
1,434,763,5,886724873
2,57,1093,3,883697352
3,455,1086,3,879109692
4,892,631,4,886609726


In [50]:
train_matrix = train_df.pivot_table(index='user', columns='item', values='rating').fillna(0)

similarity = cosine_similarity(train_matrix)

In [51]:
def get_similar_users(user_id):
    similar_users =  similarity[user_id - 1]  # -1, так как индексы в pivot_table от 0 до 942, а в train_df от 1 до 943
    similar_users = list(enumerate(similar_users))
    sorted_users = sorted(similar_users, key=lambda x:x[1], reverse=True)[1:6]
    return np.array([i[0] for i in sorted_users])

def recommend(user_id):

    similar_users = get_similar_users(user_id) + 1  # +1 - вторая строка наоборот

    user_profile = train_matrix.loc[user_id]
    watched_movies = user_profile[user_profile > 0].index

    totals = train_matrix.loc[similar_users].sum(axis=0)
    amounts = train_matrix.loc[similar_users].count(axis=0).astype(float)
    averages = totals / amounts

    averages = averages.drop(watched_movies)

    averages = averages.sort_values(ascending=False)

    return averages.head(10).index.values
    

In [52]:
recommendations = {}

for row in test_df.itertuples():
    recommendations[row.user] = recommend(row.user)

In [53]:
def precision_at_k(actual, predicted, k=10):

    act_set = set(actual)
    pred_set = set(predicted[:k])
    result = len(act_set & pred_set) / min(k, len(pred_set))
    return result

def recall_at_k(actual, predicted, k=10):

    act_set = set(actual)
    pred_set = set(predicted[:k])
    result = len(act_set & pred_set) / len(act_set)
    return result


def ndcg_at_k(actual, predicted, k=10):
    dcg = ndcg_score([actual[:k]], [predicted[:k]])
    idcg = ndcg_score([actual], [actual])
    result = dcg / idcg
    return result


def evaluate(test, recommendations):

    metrics = []

    for row in test.itertuples():

        user_id = row.user
        actual = test.query(f'user == {user_id}').item.values
        predicted = recommendations[user_id]
        if actual.shape[0] >= 5: 
            p5 = precision_at_k(actual, predicted, k=5)
            r5 = recall_at_k(actual, predicted, k=5)
            ndcg5 = ndcg_at_k(actual, predicted, k=5)
    
            metrics.append([p5, r5, ndcg5])

    metrics = pd.DataFrame(metrics, columns=['P@5', 'R@5', 'NDCG@5'])

    return metrics.mean()

result_metrics = evaluate(test_df, recommendations)

In [54]:
result_metrics

P@5       0.407578
R@5       0.072824
NDCG@5    0.820033
dtype: float64